In [1]:
import numpy as np, pandas as pd, xarray as xr
from pathlib import Path
import datetime, networkx as nx, yaml, warnings, sys, logging, shutil, papermill
from disjoint_set import DisjointSet
from helper import singleglob, nxrender, Step, dict_merge, PdfWriter, DictWriter, TableWriter, TextWriter
import matplotlib.pyplot as plt
import matplotlib as mpl, seaborn as sns
from tqdm.notebook import tqdm
from typing import Literal

In [2]:
folder = Path("/home/julienb/Documents/Data/Luisa/")
scripts = ["task_graph.ipynb", "events.ipynb"]
selection: Literal["base",  "subject", "session", "existing"] = "session"
copy: Literal["force", "notexisting"]= "notexisting"
exec: Literal["force", "notexisting", "notsucess"]= "notsucess"

In [3]:
dfs = []
for i, script in enumerate(scripts):
    df = pd.DataFrame()
    if selection=="base":
        df["base_path"] = list(folder.glob("analysis"))
    elif selection=="subject":
        df["base_path"] = list(folder.glob("*/analysis"))
    elif selection=="session":
        df["base_path"] = list(folder.glob("**/Session--*/analysis"))
    elif selection=="existing":
        df["base_path"] = [f.parent for f in folder.glob(f"**/{script}")]
    dfs.append(df.assign(script=Path(script), script_index=i, script_path=df.apply(lambda row: (row["base_path"]/script), axis=1)))
tasks = pd.concat(dfs).sort_values(["base_path", "script_index"])
tasks

,base_path,script,script_index,script_path
82,/home/julienb/Documents/Data/Luisa/Rat101/opto...,task_graph.ipynb,0,/home/julienb/Documents/Data/Luisa/Rat101/opto...
82,/home/julienb/Documents/Data/Luisa/Rat101/opto...,events.ipynb,1,/home/julienb/Documents/Data/Luisa/Rat101/opto...
85,/home/julienb/Documents/Data/Luisa/Rat101/opto...,task_graph.ipynb,0,/home/julienb/Documents/Data/Luisa/Rat101/opto...
85,/home/julienb/Documents/Data/Luisa/Rat101/opto...,events.ipynb,1,/home/julienb/Documents/Data/Luisa/Rat101/opto...
79,/home/julienb/Documents/Data/Luisa/Rat101/opto...,task_graph.ipynb,0,/home/julienb/Documents/Data/Luisa/Rat101/opto...
...,...,...,...,...
13,/home/julienb/Documents/Data/Luisa/Rat246/opto...,events.ipynb,1,/home/julienb/Documents/Data/Luisa/Rat246/opto...
14,/home/julienb/Documents/Data/Luisa/Rat246/opto...,task_graph.ipynb,0,/home/julienb/Documents/Data/Luisa/Rat246/opto...
14,/home/julienb/Documents/Data/Luisa/Rat246/opto...,events.ipynb,1,/home/julienb/Documents/Data/Luisa/Rat246/opto...
12,/home/julienb/Documents/Data/Luisa/Rat246/opto...,task_graph.ipynb,0,/home/julienb/Documents/Data/Luisa/Rat246/opto...


In [4]:

if copy=="force": tasks["copy"] = True 
elif copy=="notexisting": tasks["copy"] = tasks["script_path"].apply(lambda p: not p.exists())
else: raise Exception(f"Unknown copy param {copy}")

if exec=="force": tasks["exec"] = True
elif exec=="notexisting": tasks["exec"] = tasks["script_path"].apply(lambda p: not p.with_suffix("").exists()) 
elif exec=="notsucess": tasks["exec"] = tasks["script_path"].apply(lambda p: not (p.with_suffix("")/"notebook.html").exists())
else: raise Exception(f"Unknown exec param {exec}")

display(tasks.groupby(["copy", "exec"]).count())
tasks


base_path  script  script_index  script_path
copy  exec                                               
False False         94      94            94           94
      True         164     164           164          164

,base_path,script,script_index,script_path,copy,exec
82,/home/julienb/Documents/Data/Luisa/Rat101/opto...,task_graph.ipynb,0,/home/julienb/Documents/Data/Luisa/Rat101/opto...,False,False
82,/home/julienb/Documents/Data/Luisa/Rat101/opto...,events.ipynb,1,/home/julienb/Documents/Data/Luisa/Rat101/opto...,False,False
85,/home/julienb/Documents/Data/Luisa/Rat101/opto...,task_graph.ipynb,0,/home/julienb/Documents/Data/Luisa/Rat101/opto...,False,False
85,/home/julienb/Documents/Data/Luisa/Rat101/opto...,events.ipynb,1,/home/julienb/Documents/Data/Luisa/Rat101/opto...,False,False
79,/home/julienb/Documents/Data/Luisa/Rat101/opto...,task_graph.ipynb,0,/home/julienb/Documents/Data/Luisa/Rat101/opto...,False,False
...,...,...,...,...,...,...
13,/home/julienb/Documents/Data/Luisa/Rat246/opto...,events.ipynb,1,/home/julienb/Documents/Data/Luisa/Rat246/opto...,False,True
14,/home/julienb/Documents/Data/Luisa/Rat246/opto...,task_graph.ipynb,0,/home/julienb/Documents/Data/Luisa/Rat246/opto...,False,True
14,/home/julienb/Documents/Data/Luisa/Rat246/opto...,events.ipynb,1,/home/julienb/Documents/Data/Luisa/Rat246/opto...,False,True
12,/home/julienb/Documents/Data/Luisa/Rat246/opto...,task_graph.ipynb,0,/home/julienb/Documents/Data/Luisa/Rat246/opto...,False,True


In [5]:
copy_df = tasks.loc[tasks["copy"]]
progress = tqdm(copy_df[["base_path", "script"]].iterrows(), total=len(copy_df.index))
for _, (p,s) in progress:
    shutil.copy(s, p/s)

0it [00:00, ?it/s]

In [6]:
exec_df = tasks.loc[tasks["exec"]]
base_path_grp = exec_df.groupby("base_path")
progress = tqdm(base_path_grp, total=len(base_path_grp))
errors=[]

for p, grp in progress:
    progress.set_description_str(str(Path(*p.parts[-2:-1])))
    grp = grp.sort_values("script_index")
    skip=False
    for script in grp["script"]:
        if not skip:
            progress.set_postfix_str(f"n_prev_err={len(errors)}, Executing {script}")
            (p/script.stem).mkdir(exist_ok=True, parents=True)
            try:
                papermill.execute_notebook(p/script, p/script.stem/"notebook.ipynb", cwd=p, progress_bar=False)
            except Exception as e:
                errors.append((p, script, e))
                e.add_note(f"During execution of {script} in {p}")
                skip=True
                
if len(errors) > 0:
    errors = pd.DataFrame(errors, columns=["base_path", "script", "exception"])
    display(errors[["base_path", "script"]])
    raise ExceptionGroup("Errors during execution", [e for p, s, e in errors])

  0%|          | 0/84 [00:00<?, ?it/s]

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


NameError: name 's' is not defined